<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/PIXEL-PIONEERS-TUTORIALS/blob/main/Retrieval_Evaluator_and_Action_Trigger_for_Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The trigger action RAG (Retrieval-Augmented Generation) builds on top of regular RAG by introducing a retrieval evaluator and a mechanism to trigger different actions based on the relevance scores of the retrieved documents. This extension aims to improve the quality and relevance of the generated outputs by selectively using the retrieved documents based on their relevance to the input query.**

Here's how the trigger action RAG extends the regular RAG approach:

1. **Retrieval Evaluator:**
   - In regular RAG, the retrieved documents are directly used for generating the output without explicitly evaluating their relevance to the input query.
   - The trigger action RAG introduces a retrieval evaluator component, which is typically a pre-trained language model (e.g., T5) fine-tuned to assess the relevance of the retrieved documents to the input query.
   - The retrieval evaluator assigns relevance scores to each retrieved document, indicating how well they match the input query.

2. **Triggered Actions:**
   - Regular RAG simply uses the retrieved documents as additional context for generating the output, regardless of their relevance.
   - The trigger action RAG defines different actions based on the relevance scores assigned by the retrieval evaluator.
   - The actions can be categorized as "Correct", "Incorrect", or "Ambiguous", depending on the relevance scores and predefined thresholds.
   - The "Correct" action is triggered when the relevance scores are high, indicating that the retrieved documents are highly relevant to the input query. In this case, the selected documents are used for generating the output.
   - The "Incorrect" action is triggered when the relevance scores are low, suggesting that the retrieved documents are not relevant to the input query. In this case, the retrieved documents are discarded, and the model may rely on its own knowledge or generate a response without using the retrieved information.
   - The "Ambiguous" action is triggered when the relevance scores are mixed or fall between the thresholds. In this case, the model may use a combination of the retrieved documents and its own knowledge to generate the output.

3. **Document Selection:**
   - Regular RAG uses all the retrieved documents for generating the output.
   - The trigger action RAG selectively uses the retrieved documents based on the triggered action and the relevance scores.
   - For the "Correct" action, only the documents with high relevance scores above a certain threshold are selected.
   - For the "Incorrect" action, no documents are selected, and the model generates the output without using the retrieved information.
   - For the "Ambiguous" action, all the retrieved documents may be used, or a subset of documents with relevance scores above a certain threshold may be selected.

By incorporating the retrieval evaluator and triggered actions, the trigger action RAG aims to improve the quality and relevance of the generated outputs. It allows the model to selectively use the retrieved documents based on their relevance to the input query, potentially reducing the reliance on irrelevant or noisy information.

The trigger action RAG can be seen as an extension of regular RAG that adds an additional layer of relevance evaluation and decision-making based on the retrieved documents. This extension can help in scenarios where the retrieved documents may not always be relevant or useful for generating the desired output, and the model needs to adapt its generation strategy accordingly.

### Document Relevance Evaluation and Action Triggering with T5

### Description:

This code demonstrates a comprehensive approach to evaluating the relevance of retrieved documents to a given question and subsequently triggering appropriate actions based on the evaluated relevance scores. It leverages the T5 (Text-to-Text Transfer Transformer) model and tokenizer for the evaluation process. The code consists of two main functions:

1. **`evaluate_retrieval`**: This function takes an input question, a list of retrieved documents, a pre-trained T5 model, and its tokenizer to evaluate the relevance of each document to the question. It does so by concatenating the question and each document, feeding this input to the T5 model, and interpreting the model's output as a relevance score (1.0 for relevant, 0.0 for not relevant, and 0.5 for ambiguous cases).

2. **`trigger_action`**: Based on the relevance scores obtained from `evaluate_retrieval`, this function decides on the appropriate action to take. It uses two thresholds: an upper threshold to trigger a "Correct" action (indicating high relevance) and a lower threshold for an "Incorrect" action (indicating low relevance). Documents scoring above the upper threshold are selected for further processing, while those below the lower threshold are discarded. Documents with scores in between are treated as ambiguous, potentially requiring additional review.

The example usage illustrates how to use these functions with a sample question and a set of retrieved documents. It shows how to obtain relevance scores and determine the next steps based on these scores, demonstrating a practical application of machine learning in information retrieval tasks.

[Paper for Retrieval Evaluator and Action Trigger for RAG](https://arxiv.org/html/2401.15884v2)



An example usage of retrieval evaluation and action trigger

In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer to use as the retrieval evaluator
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
evaluator_model = T5ForConditionalGeneration.from_pretrained(model_name)

def evaluate_retrieval(question, retrieved_docs, evaluator_model, tokenizer):

    scores = []

    for doc in retrieved_docs:
        # Concatenate question and document as input
        input_text = f"Question: {question} Document: {doc} Relevant:"
        input_ids = tokenizer.encode(input_text, return_tensors='pt')

        # Get relevance score from evaluator model
        with torch.no_grad():
            outputs = evaluator_model.generate(input_ids, max_length=5)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if decoded_output.lower() == 'true':
            score = 1.0
        elif decoded_output.lower() == 'false':
            score = 0.0
        else:
            score = 0.5  # Set a default score for unexpected output

        scores.append(score)

    return scores

def trigger_action(question, retrieved_docs, scores, upper_threshold=0.8, lower_threshold=0.2):

    if max(scores) > upper_threshold:
        # Trigger Correct action
        action = 'Correct'
        # Select documents with score > upper_threshold
        selected_docs = [doc for doc, score in zip(retrieved_docs, scores) if score > upper_threshold]
    elif max(scores) < lower_threshold:
        # Trigger Incorrect action
        action = 'Incorrect'
        selected_docs = [] # Discard retrieved docs
    else:
        # Trigger Ambiguous action
        action = 'Ambiguous'
        selected_docs = retrieved_docs # Use all retrieved docs

    return action, selected_docs

# Example usage
question = "What is Henry Feilden's occupation?"
retrieved_docs = [
    "Henry Feilden was a Conservative Party politician.",
    "Henry Master Feilden was educated at Eton College and served in the Second Boer War.",
    "The Feilden Baronetcy, of Feniscowles in Lancashire, was a title in the Baronetage of England."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(question, retrieved_docs, evaluator_model, tokenizer)

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(question, retrieved_docs, scores)

print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Triggered Action: Ambiguous
Selected Documents: ['Henry Feilden was a Conservative Party politician.', 'Henry Master Feilden was educated at Eton College and served in the Second Boer War.', 'The Feilden Baronetcy, of Feniscowles in Lancashire, was a title in the Baronetage of England.']


Fact Verification

In [3]:
claim = "The Eiffel Tower is located in Rome, Italy."
retrieved_docs = [
    "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",
    "Rome is the capital city of Italy and a special comune.",
    "The Colosseum is an oval amphitheatre in the centre of the city of Rome, Italy."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(claim, retrieved_docs, evaluator_model, tokenizer)

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(claim, retrieved_docs, scores)

print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Triggered Action: Ambiguous
Selected Documents: ['The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.', 'Rome is the capital city of Italy and a special comune.', 'The Colosseum is an oval amphitheatre in the centre of the city of Rome, Italy.']


Question Generation

In [4]:
answer = "Ernest Hemingway"
retrieved_docs = [
    "Ernest Hemingway was an American novelist, short-story writer, journalist, and sportsman.",
    "Hemingway's distinctive writing style is characterized by economy and understatement.",
    "Hemingway published seven novels, six short-story collections, and two nonfiction works during his lifetime."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(answer, retrieved_docs, evaluator_model, tokenizer)

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(answer, retrieved_docs, scores)

print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Triggered Action: Ambiguous
Selected Documents: ['Ernest Hemingway was an American novelist, short-story writer, journalist, and sportsman.', "Hemingway's distinctive writing style is characterized by economy and understatement.", 'Hemingway published seven novels, six short-story collections, and two nonfiction works during his lifetime.']


Abstractive Summarization





In [6]:
document = "The Mona Lisa is a 16th-century portrait painted in oil on a poplar panel by Leonardo da Vinci. It is probably the world's most famous painting, and is one of the most parodied works of art. The painting is a portrait of Lisa Gherardini, the wife of Francesco del Giocondo, and is in oil on a white Lombardy poplar panel. It had been believed to have been painted between 1503 and 1506; however, Leonardo may have continued working on it as late as 1517. It was acquired by King Francis I of France and is now the property of the French Republic. It has been on permanent display at the Louvre in Paris since 1797."
retrieved_docs = [
    "The Mona Lisa is a half-length portrait painting by Italian artist Leonardo da Vinci.",
    "The painting is a portrait of Lisa Gherardini, the wife of Francesco del Giocondo.",
    "The Mona Lisa is one of the most valuable paintings in the world."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(document, retrieved_docs, evaluator_model, tokenizer)

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(document, retrieved_docs, scores)

print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Triggered Action: Ambiguous
Selected Documents: ['The Mona Lisa is a half-length portrait painting by Italian artist Leonardo da Vinci.', 'The painting is a portrait of Lisa Gherardini, the wife of Francesco del Giocondo.', 'The Mona Lisa is one of the most valuable paintings in the world.']


Trigger - Ambiguous, Correct vs Incorrect

In this example, we use the same input document about "The Starry Night" painting and provide a mix of relevant and irrelevant retrieved documents. We manually set the scores to demonstrate all three triggered actions:

The first retrieved document is highly relevant, so it gets a score of 0.9.
The second retrieved document is irrelevant, so it gets a score of 0.1.
The third retrieved document is somewhat relevant, so it gets a score of 0.6.
Based on these scores and the defined thresholds (upper_threshold=0.8 and lower_threshold=0.2), the triggered action will be "Ambiguous" because:

The maximum score (0.9) is greater than the upper_threshold, triggering the "Correct" action.
However, there is also a score (0.1) that is lower than the lower_threshold, which would trigger the "Incorrect" action.
Since there are conflicting scores, the "Ambiguous" action is triggered as a fallback.
The selected documents will include all the retrieved documents since the action is "Ambiguous".

When you run this code, it will output the triggered action as "Ambiguous" and the selected documents, demonstrating how the code handles a mix of relevant and irrelevant retrieved documents.


In [8]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer to use as the retrieval evaluator
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
evaluator_model = T5ForConditionalGeneration.from_pretrained(model_name)

def evaluate_retrieval(document, retrieved_docs, evaluator_model, tokenizer):
    '''
    Evaluate the relevance of retrieved documents to the input document.

    Args:
        document (str): The input document
        retrieved_docs (list): List of retrieved documents
        evaluator_model: The T5 model used as the retrieval evaluator
        tokenizer: The T5 tokenizer

    Returns:
        scores (list): Relevance scores between 0 and 1 for each retrieved doc
    '''
    scores = []

    for doc in retrieved_docs:
        # Concatenate document and retrieved doc as input
        input_text = f"Document: {document} Retrieved: {doc} Relevant:"
        input_ids = tokenizer.encode(input_text, return_tensors='pt')

        # Get relevance score from evaluator model
        with torch.no_grad():
            outputs = evaluator_model.generate(input_ids, max_length=5)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if decoded_output.lower() == 'true':
            score = 1.0
        elif decoded_output.lower() == 'false':
            score = 0.0
        else:
            score = 0.5  # Set a default score for unexpected output

        scores.append(score)

    return scores

def trigger_action(document, retrieved_docs, scores, upper_threshold=0.8, lower_threshold=0.2):
    '''
    Trigger an appropriate retrieval action based on evaluator scores.

    Args:
        document (str): The input document
        retrieved_docs (list): List of retrieved documents
        scores (list): Relevance scores for each retrieved doc
        upper_threshold (float): Threshold for triggering Correct action
        lower_threshold (float): Threshold for triggering Incorrect action

    Returns:
        action (str): The triggered action - 'Correct', 'Incorrect' or 'Ambiguous'
        selected_docs (list): The documents selected based on the action
    '''
    if max(scores) > upper_threshold:
        # Trigger Correct action
        action = 'Correct'
        # Select documents with score > upper_threshold
        selected_docs = [doc for doc, score in zip(retrieved_docs, scores) if score > upper_threshold]
    elif max(scores) < lower_threshold:
        # Trigger Incorrect action
        action = 'Incorrect'
        selected_docs = [] # Discard retrieved docs
    else:
        # Trigger Ambiguous action
        action = 'Ambiguous'
        selected_docs = retrieved_docs # Use all retrieved docs

    return action, selected_docs

# Example - Demonstrating all three triggered actions
document = "The Starry Night is an oil painting by Dutch Post-Impressionist painter Vincent van Gogh."
retrieved_docs = [
    "The Starry Night is an oil on canvas painting by Dutch Post-Impressionist painter Vincent van Gogh.",
    "The Mona Lisa is a half-length portrait painting by Italian artist Leonardo da Vinci.",
    "Van Gogh painted The Starry Night in June 1889 while he was a patient in the Saint-Paul-de-Mausole asylum in France."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(document, retrieved_docs, evaluator_model, tokenizer)

# Manually set scores for demonstration purposes
scores = [0.9, 0.1, 0.6]

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(document, retrieved_docs, scores)

print("Example - Demonstrating all three triggered actions:")
print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Example - Demonstrating all three triggered actions:
Triggered Action: Correct
Selected Documents: ['The Starry Night is an oil on canvas painting by Dutch Post-Impressionist painter Vincent van Gogh.']


In [9]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained T5 model and tokenizer to use as the retrieval evaluator
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
evaluator_model = T5ForConditionalGeneration.from_pretrained(model_name)

def evaluate_retrieval(document, retrieved_docs, evaluator_model, tokenizer):
    '''
    Evaluate the relevance of retrieved documents to the input document.

    Args:
        document (str): The input document
        retrieved_docs (list): List of retrieved documents
        evaluator_model: The T5 model used as the retrieval evaluator
        tokenizer: The T5 tokenizer

    Returns:
        scores (list): Relevance scores between 0 and 1 for each retrieved doc
    '''
    scores = []

    for doc in retrieved_docs:
        # Concatenate document and retrieved doc as input
        input_text = f"Document: {document} Retrieved: {doc} Relevant:"
        input_ids = tokenizer.encode(input_text, return_tensors='pt')

        # Get relevance score from evaluator model
        with torch.no_grad():
            outputs = evaluator_model.generate(input_ids, max_length=5)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if decoded_output.lower() == 'true':
            score = 1.0
        elif decoded_output.lower() == 'false':
            score = 0.0
        else:
            score = 0.5  # Set a default score for unexpected output

        scores.append(score)

    return scores

def trigger_action(document, retrieved_docs, scores, upper_threshold=0.8, lower_threshold=0.2):
    '''
    Trigger an appropriate retrieval action based on evaluator scores.

    Args:
        document (str): The input document
        retrieved_docs (list): List of retrieved documents
        scores (list): Relevance scores for each retrieved doc
        upper_threshold (float): Threshold for triggering Correct action
        lower_threshold (float): Threshold for triggering Incorrect action

    Returns:
        action (str): The triggered action - 'Correct', 'Incorrect' or 'Ambiguous'
        selected_docs (list): The documents selected based on the action
    '''
    if max(scores) > upper_threshold:
        # Trigger Correct action
        action = 'Correct'
        # Select documents with score > upper_threshold
        selected_docs = [doc for doc, score in zip(retrieved_docs, scores) if score > upper_threshold]
    elif max(scores) < lower_threshold:
        # Trigger Incorrect action
        action = 'Incorrect'
        selected_docs = [] # Discard retrieved docs
    else:
        # Trigger Ambiguous action
        action = 'Ambiguous'
        selected_docs = retrieved_docs # Use all retrieved docs

    return action, selected_docs

# Example - Demonstrating the "Incorrect" triggered action
document = "The Starry Night is an oil painting by Dutch Post-Impressionist painter Vincent van Gogh."
retrieved_docs = [
    "The Mona Lisa is a half-length portrait painting by Italian artist Leonardo da Vinci.",
    "The Last Supper is a late 15th-century mural painting by Italian artist Leonardo da Vinci.",
    "The Girl with a Pearl Earring is an oil painting by Dutch Golden Age painter Johannes Vermeer."
]

# Get relevance scores from retrieval evaluator
scores = evaluate_retrieval(document, retrieved_docs, evaluator_model, tokenizer)

# Manually set scores for demonstration purposes
scores = [0.1, 0.05, 0.15]

# Trigger appropriate action based on scores
action, selected_docs = trigger_action(document, retrieved_docs, scores)

print("Example - Demonstrating the 'Incorrect' triggered action:")
print(f"Triggered Action: {action}")
print(f"Selected Documents: {selected_docs}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Example - Demonstrating the 'Incorrect' triggered action:
Triggered Action: Incorrect
Selected Documents: []
